In [1]:
import pickle

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        return pickle.load(f)

extracted_path = "/kaggle/input/training-data/Training"  # Adjust if needed

# Load datasets (Adjust file paths accordingly)
X1_train = load_pickle(f"{extracted_path}/X1_train.pkl")
X1_val = load_pickle(f"{extracted_path}/X1_val.pkl")
X1_sub = load_pickle(f"{extracted_path}/X1_sub.pkl")
X2_train = load_pickle(f"{extracted_path}/X2_train_fixed.pkl")
X2_val = load_pickle(f"{extracted_path}/X2_val_fixed.pkl")
X2_sub = load_pickle(f"{extracted_path}/X2_sub_fixed.pkl")
y_train = load_pickle(f"{extracted_path}/y_train.pkl")
y_val = load_pickle(f"{extracted_path}/y_val.pkl")

print("Datasets loaded successfully!")

Datasets loaded successfully!


# Helper Functions and libraries

In [2]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate,GlobalAveragePooling2D,  BatchNormalization
from tensorflow.keras.models import Model

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [4]:
def evaluate_model(model, X1_test, X2_test, y_test):
    """
    Evaluate a multi-input CNN model using R² score, MAE, and RMSE.

    Parameters:
        model (tf.keras.Model): Trained CNN model.
        X1_test (np.ndarray): Test data for the first input (e.g., ROI from landsat_result_df).
        X2_test (np.ndarray): Test data for the second input (e.g., ROI from result_df).
        y_test (np.ndarray): True target values (e.g., UHI index).

    Returns:
        dict: A dictionary containing R² score, MAE, and RMSE.
    """

    # Make predictions
    y_pred = model.predict([X1_test, X2_test])

    # Flatten predictions and true values if necessary
    y_pred = y_pred.flatten()
    #y_test = y_test.flatten()

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Return results as a dictionary
    return {
        "R² Score": r2,
        "MAE": mae,
        "RMSE": rmse
    }

In [5]:
def se_block(input_tensor, ratio=8):
    channels = input_tensor.shape[-1]  # Get the number of filters in the input

    squeeze = layers.GlobalAveragePooling2D()(input_tensor)  # Squeeze: Reduce spatial dimensions
    excitation = layers.Dense(channels // ratio, activation="relu")(squeeze)  # Bottleneck
    excitation = layers.Dense(channels, activation="sigmoid")(excitation)  # Scale factor
    excitation = layers.Reshape((1, 1, channels))(excitation)  # Reshape for broadcasting

    return layers.Multiply()([input_tensor, excitation])

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Multiply, Reshape


class CoordinateAttention(Layer):
    def __init__(self, reduction=32, **kwargs):
        super(CoordinateAttention, self).__init__(**kwargs)
        self.reduction = reduction

    def build(self, input_shape):
        _, h, w, c = input_shape
        reduced_channels = max(1, c // self.reduction)

        # Coordinate attention splits across H and W
        self.conv1x1_h = Conv2D(reduced_channels, (1, 1), activation='relu', padding='same')
        self.conv1x1_w = Conv2D(reduced_channels, (1, 1), activation='relu', padding='same')
        self.conv1x1_out = Conv2D(c, (1, 1), activation='sigmoid', padding='same')

    def call(self, x):
        h_pool = tf.reduce_mean(x, axis=2, keepdims=True)  # Pooling along width
        w_pool = tf.reduce_mean(x, axis=1, keepdims=True)  # Pooling along height

        h_out = self.conv1x1_h(h_pool)
        w_out = self.conv1x1_w(w_pool)

        attention = self.conv1x1_out(h_out + w_out)  # Merge attention
        return x * attention  # Apply attention weights

    # Add a get_config method for serialization
    def get_config(self):
        config = super(CoordinateAttention, self).get_config()
        config.update({'reduction': self.reduction})
        return config

In [8]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense, Concatenate, GlobalAveragePooling2D, Add
from tensorflow.keras.models import Model
import tensorflow as tf

# Assuming CoordinateAttention and se_block are defined elsewhere
# Define input shapes
input_shape_1 = (25, 25, 6)  # X1: landsat_result_df
input_shape_2 = (25, 25, 6)  # X2: result_df

# Input layers
input_1 = Input(shape=input_shape_1, name='input_1')
input_2 = Input(shape=input_shape_2, name='input_2')

# Feature extractor for input_1 (X1: 47x importance)
x1 = Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(input_1)
x1 = BatchNormalization()(x1)
x1 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x1)
x1 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x1)
x1 = CoordinateAttention(reduction=8)(x1)  # Tighter focus for X1
x1_res = Conv2D(64, kernel_size=(1, 1), activation='relu', padding='same')(x1)  # Residual path
x1 = se_block(x1_res,ratio = 8)  # Stronger channel weighting
x1 = Add()([x1, x1_res])  # Residual connection
x1 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x1)  # Extra depth
x1 = GlobalAveragePooling2D()(x1)

# Feature extractor for input_2 (X2: weaker signal)
x2 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(input_2)
x2 = BatchNormalization()(x2)
x2 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x2)
x2 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x2)
x2 = CoordinateAttention(reduction=16)(x2)  # Lighter attention
x2_res = Conv2D(32, kernel_size=(1, 1), activation='relu', padding='same')(x2)
x2 = se_block(x2_res, ratio=16)
x2 = Add()([x2, x2_res])
x2 = GlobalAveragePooling2D()(x2)

# Weighted fusion
x1_weighted = Dense(64, activation='relu', kernel_initializer='he_normal')(x1)
x2_weighted = Dense(32, activation='relu', kernel_initializer='he_normal')(x2)
merged = Concatenate()([x1_weighted, x2_weighted])
merged = Dense(64, activation='relu')(merged)  # Interaction layer

# Output head
x = Dense(32, activation='relu')(merged)
x = Dense(16, activation='relu')(x)
output = Dense(1, activation='linear', name='output')(x)

# Model
model = Model(inputs=[input_1, input_2], outputs=output)

# Optimizer and compile
optimizer = tf.keras.optimizers.AdamW(learning_rate=5e-4, weight_decay=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae', tf.keras.metrics.R2Score()])

# Summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)      │ (None, 25, 25, 6)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 25, 25, 512)    │         28,160 │ input_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_2 (InputLayer)      │ (None, 25, 25, 6)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 25, 25, 512)    │          2,048 │ conv2d_15[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_23 (Conv2D)        │ (None, 25, 25, 256)    │         14,080 │ input_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 25, 25, 256)    │      1,179,904 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 25, 25, 256)    │          1,024 │ conv2d_23[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 25, 25, 128)    │        295,040 │ conv2d_16[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_24 (Conv2D)        │ (None, 25, 25, 128)    │        295,040 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ coordinate_attention_2    │ (None, 25, 25, 128)    │          6,304 │ conv2d_17[0][0]        │
│ (CoordinateAttention)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_25 (Conv2D)        │ (None, 25, 25, 64)     │         73,792 │ conv2d_24[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 25, 25, 64)     │          8,256 │ coordinate_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ coordinate_attention_3    │ (None, 25, 25, 64)     │            840 │ conv2d_25[0][0]        │
│ (CoordinateAttention)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 64)             │              0 │ conv2d_21[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_29 (Conv2D)        │ (None, 25, 25, 32)     │          2,080 │ coordinate_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 8)              │            520 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 32)             │              0 │ conv2d_29[0][0]        │
│ (GlobalAveragePooling

 Total params: 1,958,803 (7.47 MB)

 Trainable params: 1,957,267 (7.47 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=50,
    restore_best_weights=True
)

# Callbacks
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Updated extension for TF 2.16+
    monitor='val_r2_score',       # Monitor validation R²
    save_best_only=True,          # Save only the best model
    mode='max',                   # Maximize R²
    verbose=1
)

# Fit the model
history = model.fit(
    [X1_train, X2_train], y_train,                   
    validation_data=([X1_val, X2_val], y_val),       
    epochs=150,
    batch_size=8,
    callbacks=[checkpoint, early_stopping],                         
    verbose=1
)


Epoch 1/150
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0226 - mae: 0.0534 - r2_score: -107.3160
Epoch 1: val_r2_score improved from -inf to 0.33213, saving model to best_model.keras
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 56s 32ms/step - loss: 0.0226 - mae: 0.0534 - r2_score: -107.2336 - val_loss: 1.7532e-04 - val_mae: 0.0107 - val_r2_score: 0.3321
Epoch 2/150
1121/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.5819e-04 - mae: 0.0171 - r2_score: -0.7759
Epoch 2: val_r2_score did not improve from 0.33213
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - loss: 4.5804e-04 - mae: 0.0171 - r2_score: -0.7753 - val_loss: 2.7027e-04 - val_mae: 0.0136 - val_r2_score: -0.0296
Epoch 3/150
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.1775e-04 - mae: 0.0144 - r2_score: -0.1750
Epoch 3: val_r2_score did not improve from 0.33213
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - loss: 3.1772e-04 - mae: 0.0144 - r2_score: -0.1749 - val_loss: 6.3117e-04 - val_mae: 0.0228 - val_r2_score: -1.4044
E

In [11]:
evaluate_model(model, X1_val, X2_val, y_val)

71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step


{'R² Score': 0.9521358847226221,
 'MAE': 0.0024966588012531707,
 'RMSE': 0.003547316263585546}

In [12]:
sub_df = pd.read_csv('/kaggle/input/sub-uhi-csv/Submission_template_UHI2025-v2.csv')

final_predictions = model.predict([X1_sub, X2_sub])
final_prediction_series = pd.Series(final_predictions.flatten())

submission_df = pd.DataFrame({'Longitude':sub_df['Longitude'].values, 'Latitude':sub_df['Latitude'].values, 'UHI Index':final_prediction_series.values})

#Dumping the predictions into a csv file.
submission_df.to_csv("new_coor_submission_v3.csv",index = False)

33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step


In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler


# Learning rate scheduler (cosine decay with warmup)
def lr_schedule(epoch, lr):
    warmup_epochs = 5
    total_epochs = 100
    initial_lr = 5e-4
    if epoch < warmup_epochs:
        return initial_lr * (epoch + 1) / warmup_epochs  # Linear warmup
    else:
        # Cosine decay to 1e-5
        decay_steps = total_epochs - warmup_epochs
        cosine_decay = 0.5 * (1 + np.cos(np.pi * (epoch - warmup_epochs) / decay_steps))
        return initial_lr * cosine_decay + 1e-5 * (1 - cosine_decay)

# Callbacks
checkpoint = ModelCheckpoint(
    filepath='best_model_finetuned.keras',
    monitor='val_r2_score',
    save_best_only=True,
    mode='max',
    verbose=1
)
lr_scheduler = LearningRateScheduler(lr_schedule)

# Fit the model
history = model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_val, X2_val], y_val),
    epochs=100,
    batch_size=8,
    callbacks=[checkpoint, lr_scheduler],
    verbose=1
)

Epoch 1/100
1121/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.2659e-06 - mae: 0.0015 - r2_score: 0.9838
Epoch 1: val_r2_score improved from -inf to 0.95272, saving model to best_model_finetuned.keras
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - loss: 4.2655e-06 - mae: 0.0015 - r2_score: 0.9838 - val_loss: 1.2413e-05 - val_mae: 0.0025 - val_r2_score: 0.9527 - learning_rate: 1.0000e-04
Epoch 2/100
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 4.2643e-06 - mae: 0.0015 - r2_score: 0.9836
Epoch 2: val_r2_score did not improve from 0.95272
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - loss: 4.2646e-06 - mae: 0.0015 - r2_score: 0.9836 - val_loss: 1.2547e-05 - val_mae: 0.0025 - val_r2_score: 0.9522 - learning_rate: 2.0000e-04
Epoch 3/100
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 5.0345e-06 - mae: 0.0017 - r2_score: 0.9802
Epoch 3: val_r2_score did not improve from 0.95272
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - loss: 5.0348e-06 - mae: 0.0017 - r2_score: 0.9802 - v

In [15]:
evaluate_model(model, X1_val, X2_val, y_val)

71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


{'R² Score': 0.9559783698427292,
 'MAE': 0.0022689583447073904,
 'RMSE': 0.0034019502055302113}

In [16]:
final_predictions = model.predict([X1_sub, X2_sub])
final_prediction_series = pd.Series(final_predictions.flatten())

submission_df = pd.DataFrame({'Longitude':sub_df['Longitude'].values, 'Latitude':sub_df['Latitude'].values, 'UHI Index':final_prediction_series.values})

#Dumping the predictions into a csv file.
submission_df.to_csv("weighted_submission_v3.csv",index = False)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


# Fine tune model

In [7]:
loaded_model = tf.keras.models.load_model(
    '/kaggle/input/coor_attention_cnn/tensorflow2/200epoch_untuned/1/best_model (1).keras',
    custom_objects={'CoordinateAttention': CoordinateAttention}
)

loaded_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)      │ (None, 25, 25, 6)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_2 (InputLayer)      │ (None, 25, 25, 6)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 25, 25, 512)    │         12,800 │ input_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 25, 25, 512)    │         12,800 │ input_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 25, 25, 512)    │          2,048 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 25, 25, 512)    │          2,048 │ conv2d_7[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 25, 25, 256)    │      1,179,904 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 25, 25, 256)    │      1,179,904 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 25, 25, 128)    │        295,040 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 25, 25, 128)    │        295,040 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ coordinate_attention      │ (None, 25, 25, 128)    │          1,672 │ conv2d_2[0][0]         │
│ (CoordinateAttention)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ coordinate_attention_1    │ (None, 25, 25, 128)    │          1,672 │ conv2d_9[0][0]         │
│ (CoordinateAttention)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 25, 25, 64)     │          8,256 │ coordinate_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 25, 25, 64)     │          8,256 │ coordinate_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 64)             │              0 │ conv2d_6[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 64)             │              0 │ conv2d_13[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)        

 Total params: 9,081,445 (34.64 MB)

 Trainable params: 3,026,465 (11.55 MB)

 Non-trainable params: 2,048 (8.00 KB)

 Optimizer params: 6,052,932 (23.09 MB)

In [8]:
evaluate_model(loaded_model, X1_val, X2_val, y_val)

71/71 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step


{'R² Score': 0.9428163783869361,
 'MAE': 0.002901478728689287,
 'RMSE': 0.0038773117447532773}

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.1,          # Reduce LR by a factor of 10
    patience=10,         # Wait 10 epochs before reducing LR
    min_lr=1e-6,         # Minimum learning rate
    verbose=1            # Print updates
)

# Add callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model_lr_scheduler.keras', save_best_only=True),
    lr_scheduler
]

# Train the model
history = loaded_model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_val, X2_val], y_val),
    epochs=200,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 50s 128ms/step - loss: 5.3030e-06 - mae: 0.0025 - val_loss: 1.2962e-05 - val_mae: 0.0040 - learning_rate: 0.0010
Epoch 2/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 88ms/step - loss: 5.3185e-06 - mae: 0.0026 - val_loss: 2.7558e-05 - val_mae: 0.0060 - learning_rate: 0.0010
Epoch 3/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 26s 93ms/step - loss: 5.4886e-06 - mae: 0.0026 - val_loss: 1.1061e-05 - val_mae: 0.0036 - learning_rate: 0.0010
Epoch 4/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 90ms/step - loss: 9.1420e-06 - mae: 0.0034 - val_loss: 1.2463e-05 - val_mae: 0.0039 - learning_rate: 0.0010
Epoch 5/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 90ms/step - loss: 6.3783e-06 - mae: 0.0028 - val_loss: 9.0951e-06 - val_mae: 0.0032 - learning_rate: 0.0010
Epoch 6/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 90ms/step - loss: 4.6620e-06 - mae: 0.0023 - val_loss: 1.8217e-05 - val_mae: 0.0050 - learning_rate: 0.0010
Epoch 7/200
281/281 ━━━━━━━━━━━━━━━━━━━━ 26s 91ms/step - loss: 4.6492e-06 - mae: 

In [20]:
evaluate_model(loaded_model, X1_val, X2_val, y_val)

71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step


{'R² Score': 0.9586692437199564,
 'MAE': 0.002373432901225808,
 'RMSE': 0.0032963367189493636}

In [21]:
sub_df = pd.read_csv('/kaggle/input/sub-uhi-csv/Submission_template_UHI2025-v2.csv')

final_predictions = loaded_model.predict([X1_sub, X2_sub])
final_prediction_series = pd.Series(final_predictions.flatten())

submission_df = pd.DataFrame({'Longitude':sub_df['Longitude'].values, 'Latitude':sub_df['Latitude'].values, 'UHI Index':final_prediction_series.values})

#Dumping the predictions into a csv file.
submission_df.to_csv("new_coor_ft_submission_v3.csv",index = False)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


In [18]:
loaded_model.save('/kaggle/working/fine_tuned_new_coor.h5')